In [1]:
model = "output"
cascade = "haarcascade_frontalface_default.xml"

In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2


In [3]:
# load the face detector cascade and smile detector CNN
detector = cv2.CascadeClassifier(cascade)
model = load_model(model)

2021-11-03 22:47:18.239396: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
camera = cv2.VideoCapture(0)


# keep looping
while True:
    # grab the current frame
    (grabbed, frame) = camera.read()

    # reszie the frame, convert it to grayscale, then clone the
    # original frame so we can draw on it later in the program
    frame = imutils.resize(frame, width = 300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frameClone = frame.copy()

    # detect faces in the input frame, then clone the frame so
    # that we can draw on it
    rects = detector.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,
        minSize = (30, 30), flags = cv2.CASCADE_SCALE_IMAGE)

    for (fX, fY, fW, fH) in rects:
        # extract the ROI of the face from the grayscale image,
        # resize it to a fixed 28x28 pixels, and then prepare the
        # ROI for classification via CNN
        roi = gray[fY: fY + fH, fX: fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)

        # determine the probabilities of both "smiling" and "not similing"
        # then set the label accordingly
        (notSmiling, smiling) = model.predict(roi)[0]
        label = "Smiling" if smiling > notSmiling else "Not Smiling"

        # display the label and bounding box rectangle on the output frame
        cv2.putText(frameClone, label, (fX, fY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
            (0, 0, 255), 2)

    # show our detected faces along with smiling/not smiling labels
    cv2.imshow("Face", frameClone)

    # if 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

2021-11-03 22:47:20.366118: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


KeyboardInterrupt: 